Import librairies

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import tqdm
import atlite

from shapely.geometry import mapping


# EirGrid

## Locations of farms

Read capacity of wind farms

In [ ]:
df_capacity = pd.read_csv('../Data_Final/EirGrid/capacity_wind_9224_eir.csv',
                               index_col = 2,
                               parse_dates=True)

Rename the columns to be correctly read by Atlite

In [ ]:
df_capacity = df_capacity.dropna().rename(columns={'Capacity (MW)':'capacity', 'Latitude':'y', 'Longitude':'x'})
df_capacity = df_capacity[df_capacity.index < '2024'] # Make sure to remove all data before 2024

# Atlite

## Create Cutout for Ireland

Define function to create a time dependent layout of the capacity from the dataframe

In [ ]:
def get_time_dependent_capacity_distribution(cutout: atlite.Cutout, df_capacity: pd.DataFrame):
    capacity_layout = cutout.data['roughness'].copy()
    capacity_layout.name = 'Capacity'
    capacity_layout[:,:,:] = 0.

    # Iterate over all capacity installations
    for idx, row in tqdm(df_capacity.reset_index().iterrows(), total= df_capacity.shape[0]):
        cap = row['capacity'] # Capacity values
        df_capacity_i = pd.DataFrame([row])
        layout = cutout.layout_from_capacity_list(df_capacity_i, col="capacity")

        capacity_layout[capacity_layout['time'] >= row['Connection date']] += layout

    return capacity_layout

In [ ]:
ds_u100 = xr.open_dataset('~/Documents/ERA5/u_100_hourly_7923.nc')
ds_v100 = xr.open_dataset('~/Documents/ERA5/v_100_hourly_7923.nc')
ds_rouhgness = xr.open_dataset('~/Documents/ERA5/forecast_surface_hourly_7923.nc')

Some variables need to be formated

In [ ]:
ds_clim = xr.merge([ds_u100, ds_v100, ds_rouhgness])
ds_clim = ds_clim.sel(time=slice("1979-01-01", "2023-12-31"))

Loads the climate variables in Atlite

This can only be done via the function get_cutout_from_era5_data which is not originally in the Atlite scripts but has been added by the authors

In [ ]:
cutout = atlite.cutout.get_cutout_from_era5_data('path', ds_clim, ['wind'])

### Create a layout of the 2023 wind farms

Since we have the cutout and a fixed distribution of capacities, we can use the internal atlite function.

In [ ]:
df_layout = cutout.layout_from_capacity_list(df_capacity)

### Load the wind turbine information

The turbine selected for this purpose is the one that was found to be the best fit for the EirGrid data in the verification phase: the Enercon E112 4500 with 0.30w smoothing obtained from the renewables.ninja database.

In [ ]:
df_turbines = pd.read_csv('../../Data/renewables_ninja/Wind Turbine Power Curves ~ 5 (0.01ms with 0.30 w smoother).csv')
turbine_ninja_030smoothing = {'hub_height':100,
                              'P':1.,
                              'V':df_turbines['data$speed'].values,
                              'POW':df_turbines['Enercon.E112.4500'].values
                             }

### Get wind generation

In [ ]:
df_gen_atlite = cutout.wind(turbine = turbine_ninja_030smoothing,
                         layout = df_layout,
                         show_progress = True,
                         add_cutout_windspeed = True)

[########################################] | 100% Completed | 3.76 s


In [ ]:
df_cf_atlite = df_gen_atlite / df_capacity['Capacity'].sum()

# C3S-E G

In [ ]:
da_energy = 

In [ ]:
def get_cf_series_c3se_gridded(da_gridded: xr.DataArray, df_capacity: pd.DataFrame):
    lats = da_gridded['latitude'].values
    lons = da_gridded['longitude'].values

    summed_time_series = np.zeros(da_gridded['time'].shape)

    total_cap = 0.
    # Iterate over all capacity installations
    for idx, row in df_capacity.reset_index().iterrows():
        x = row['x']
        y = row['y']
        cap = row['capacity']

    # Find the nearest lat/lon point
        dif_min_lon = np.argmin(abs(lons-x))
        dif_min_lat = np.argmin(abs(lats-y))

        time_series = da_gridded[:, dif_min_lat, dif_min_lon]

        pairs_list = [[0,0], [0,1], [1,0], [1,1], [2,0], [0,2], [2,1], [1,2]]
        if np.isnan(time_series).sum() > 2:
            for pair in pairs_list:
                dif_min_lon = np.argsort(abs(lons-x))[pair[0]]
                dif_min_lat = np.argsort(abs(lats-y))[pair[1]]
                time_series = da_gridded[:, dif_min_lat, dif_min_lon]
                if np.isnan(time_series).sum() <= 2:
                    break
        try:
            summed_time_series += cap*time_series
        except NotImplementedError:
            summed_time_series += cap*time_series.values

    # Divide the total time series by the total IC to go back to CF
    cf_time_series = summed_time_series/df_capacity['capacity'].sum()
    return cf_time_series

In [ ]:
array_cf_c3se_grid = get_cf_series_c3se_gridded(da_gridded=da_energy, df_capacity=df_capacity)

In [ ]:
df_cf_c3se_grid = pd.Series(data=array_cf_c3se_grid, index=da_energy['time'])

# C3S-E N

In [ ]:
df_nat = pd.read_csv('../../Data/C3S-E/c3se_onshorewind_capacityfactor_national.csv',
                         skiprows = 52,
                         usecols = [0,18], # Data for IE
                         index_col = 0,
                         parse_dates = True)

In [ ]:
df_sub = pd.read_csv('../../Data/C3S-E/c3se_onshorewind_capacityfactor_subnational.csv',
                     skiprows=52,
                     usecols=[0,350], # Data for NI
                     index_col=0,
                     parse_dates = True
)

In [ ]:
da_land_sea_mask = xr.open_dataarray('../../Data/era5_land_sea_mask.nc')

https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts#nuts21

In [ ]:
boundaries = gpd.read_file('../../Data/NUTS_RG_01M_2021_4326/NUTS_RG_01M_2021_4326.shp', crs="epsg:4326")
boundaries_ie = boundaries[boundaries['NUTS_ID']=='IE']
boundaries_ni = boundaries[boundaries['NUTS_ID']=='UKN']

In [ ]:
da_land_sea_mask.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
da_land_sea_mask.rio.write_crs("epsg:4326", inplace=True)

In [ ]:
clipped_ie = da_land_sea_mask.rio.clip(boundaries_ie.geometry.apply(mapping), boundaries_ie.crs, drop=False)
clipped_ni = da_land_sea_mask.rio.clip(boundaries_ni.geometry.apply(mapping), boundaries_ni.crs, drop=False)

In [ ]:
df_cf_c3se_nat = (df_nat['IE']*clipped_ie.sum().item() + df_sub['UKN0']*clipped_ni.sum().item())/(clipped_ie.sum().item() + clipped_ni.sum().item())

# Save Data